In [1]:
import pandas as pd
import numpy as np
import os
from nice_funcs.indicators import CreateRandomPrtf,EWMA,MACD,RSI,NormalizeWindow
from ambiente import TradingEnv


def GetIndex(*args):
  indicators = [*args]
  index_init = set(indicators[0].index)
  for ind_ in indicators:
    index_init = index_init & set(ind_.index)
  
  idx_date = min(index_init)
  new_index_indicators = []
  for ind_ in indicators:
    ind_['Cash'] = 0 
    new_index_indicators.append(ind_[idx_date:])
  return new_index_indicators
        


# %%
path_diario = './assets/1d/'
ativos = os.listdir(path_diario)

ativosOHLC = {}
for ativo in ativos:
    ativosOHLC[ativo.replace('.xlsx','')] = \
        pd.read_excel(os.path.join(path_diario,ativo),index_col=0)
    

close_prices = {}
for k in ativosOHLC.keys():
  close_prices[k] = ativosOHLC[k].Close


df_fechamento = pd.DataFrame(close_prices).iloc[:-360]
normalized_fech = df_fechamento.apply(lambda row: NormalizeWindow(row)).dropna()
macd = normalized_fech.apply(lambda row: MACD(row)[0]).dropna()
rsi = normalized_fech.apply(lambda row: RSI(row)).dropna()
ewma_diff = normalized_fech.apply(lambda row: EWMA(row,20) - EWMA(row,5)).dropna()






df_fechamento,normalized_fech,macd,rsi,ewma_diff =  GetIndex(df_fechamento,normalized_fech, macd, rsi, ewma_diff)


env = TradingEnv(df_fechamento,[normalized_fech,macd,rsi,ewma_diff])

In [2]:
normalized_fech

,ADAUSDT,BNBUSDT,BTCUSDT,ETHUSDT,LTCUSDT,XRPUSDT,Cash
Date,,,,,,,
2018-06-18,-1.103457,1.411699,-0.768851,-0.732371,-1.070490,-1.154540,0
2018-06-19,-0.889136,1.337789,-0.651483,-0.300377,-0.973955,-0.923059,0
2018-06-20,-0.982724,0.573641,-0.549649,-0.289839,-0.955627,-0.988063,0
2018-06-21,-1.031417,1.458236,-0.561819,-0.449708,-0.940942,-0.968987,0
2018-06-22,-1.608189,-0.168414,-1.631754,-1.562283,-1.715639,-1.574267,0
...,...,...,...,...,...,...,...
2022-08-05,0.832198,2.062444,0.627964,1.425203,1.684018,0.847826,0
2022-08-06,0.463622,1.777482,0.082091,0.958956,0.919183,0.413158,0
2022-08-07,1.333937,1.825976,0.351730,0.977090,0.953756,0.438750,0


In [3]:
import gymnasium
from ddpg_tf2 import Agent

In [4]:
agent = Agent(input_dims = env.observation_space.shape,\
   env = env, n_actions =env.action_space.shape[0])

ValueError: Please provide to Input a `shape` or a `tensor` or a `type_spec` argument. Note that `shape` does not include the batch dimension.

In [ ]:
obs,_ = env.reset()

**************************************************
Retorno 0.0


In [ ]:
agent.actor(obs)

TypeError: Exception encountered when calling layer 'actor_network' (type ActorNetwork).

'KerasTensor' object is not callable

Call arguments received by layer 'actor_network' (type ActorNetwork):
  • state=tf.Tensor(shape=(4, 7), dtype=float32)

In [ ]:
agent = Agent(input_dims = env.observation_space.shape,\
   env = env, n_actions =env.action_space.shape[0])


xxxxxx
n_games = 250
figure_file = 'plots/pendulum.png'
best_score = env.reward_range[0]
score_history = []
load_checkpoint = False

if load_checkpoint:
  n_steps = 0
  while n_steps <= agent.batch_size:
    observation, info = env.reset()
    action = env.action_space.sample()
    observation_, reward, _, done ,info = env.step(action)
    agent.remember(observation, action, reward, observation_, done)
    n_steps +=1
    agent.learn()
    agent.load_models()
    evaluate = True
else:
  evaluate = False

for i in range(n_games):
  observation, info = env.reset()
  done = False
  score = 0
  while not done:
    #print(score)
    action = agent.choose_action(observation, evaluate)
    observation_, reward, _ ,done ,info = env.step(action)
    score += reward
    agent.remember(observation, action, reward, observation_, done)
    #print(done)
    if not load_checkpoint:
      agent.learn()
    observation = observation_
  score_history.append(score)
  avg_score = np.mean(score_history[-100:])

  if avg_score > best_score:
    best_score = avg_score
    if not load_checkpoint:
      agent.save_models()

  print('episode ', i, 'score %.1f' % score,'avg score %1f' % avg_score )



NameError: name 'xxxxxx' is not defined

In [ ]:
np.random.random((64,4,7))

array([[[0.83429206, 0.8338765 , 0.97438553, ..., 0.41708012,
         0.35159969, 0.75159037],
        [0.47998931, 0.93194199, 0.71326925, ..., 0.89935925,
         0.16116067, 0.24908108],
        [0.17764474, 0.57324081, 0.71361155, ..., 0.4332446 ,
         0.87994378, 0.57805917],
        [0.27197629, 0.87788122, 0.01465485, ..., 0.17849221,
         0.41794329, 0.48224006]],

       [[0.81689864, 0.82913058, 0.94555025, ..., 0.19234891,
         0.73351895, 0.12182268],
        [0.45358521, 0.81268026, 0.27355927, ..., 0.85100526,
         0.82848608, 0.73389822],
        [0.38703887, 0.37911637, 0.05816795, ..., 0.57008557,
         0.85752692, 0.97686328],
        [0.7351837 , 0.33754916, 0.27061712, ..., 0.12274951,
         0.43914923, 0.78947648]],

       [[0.48506848, 0.86230897, 0.86226826, ..., 0.14096062,
         0.64709492, 0.36824822],
        [0.52072177, 0.22672211, 0.35574131, ..., 0.36993102,
         0.14421827, 0.68098374],
        [0.86901246, 0.28463497, 0.9

In [ ]:
s = np.random.random((64,4,7))
r = np.random.random((64,4,7))

In [ ]:
s = tf.convert_to_tensor(s)
r = tf.convert_to_tensor(r)

NameError: name 'tf' is not defined

In [ ]:
import tensorflow as tf

In [ ]:
tf.squeeze(agent.target_critic(s,r))

<tf.Tensor: shape=(64, 8), dtype=float32, numpy=
array([[-0.07394339, -0.08367679, -0.07830085, -0.08539984, -0.10564177,
        -0.11318593, -0.05505522, -0.082081  ],
       [-0.07855258, -0.06876447, -0.04077557, -0.06359087, -0.11602125,
        -0.08856397, -0.05475189, -0.10457182],
       [-0.06803718, -0.05771112, -0.1025174 , -0.07833898, -0.06571372,
        -0.07117561, -0.07816403, -0.04995837],
       [-0.08605063, -0.03954589, -0.04433327, -0.07946138, -0.06436015,
        -0.07610747, -0.02038309, -0.07262943],
       [-0.07157791, -0.07276114, -0.11429518, -0.09586858, -0.08293959,
        -0.11645608, -0.09193935, -0.08147249],
       [-0.07175484, -0.09051781, -0.05991995, -0.07500508, -0.06988765,
        -0.03612094, -0.11071692, -0.04737889],
       [-0.05600167, -0.08160199, -0.04955482, -0.10343876, -0.07866666,
        -0.07760549, -0.05472262, -0.07727581],
       [-0.07959028, -0.04476912, -0.05761933, -0.10487422, -0.08848538,
        -0.04706743, -0.0405477